In [132]:
pip install mahotas

     |████████████████████████████████| 5.7MB 4.1MB/s 


In [204]:
import numpy as np
import pandas as pd
import os
import string
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import mahotas 
import cv2
import os

In [205]:
dataset = pd.read_csv("data.csv")

In [206]:
dataset.head(5)


,Unnamed: 0,isHealthy,mean_r,mean_g,mean_b,stddev_r,stddev_g,stddev_b,contrast,correlation,inverse_difference_moments,entropy
0,0,Black_rot,109.290268,120.872437,93.792892,63.903836,51.597123,64.408851,806.727717,0.871684,0.072841,13.924102
1,0,Black_rot,128.443405,137.412827,114.801193,50.916858,44.833912,58.034531,661.424270,0.850571,0.077148,13.658209
2,0,Black_rot,105.817230,117.391693,101.533737,63.089894,52.666284,65.218182,225.235099,0.964996,0.155774,12.697456
3,0,Black_rot,115.003677,126.788773,109.580978,51.001996,41.572264,54.900597,149.451142,0.963721,0.184731,12.172536
4,0,Black_rot,117.467407,125.006973,106.502304,59.922952,46.281336,60.762472,733.955890,0.861248,0.065945,13.807278


In [185]:
#import seaborn as sns
#sns.pairplot(dataset,hue='isHealthy',palette='Dark2')

In [207]:
#Healthy
health_list = []
for d in dataset['isHealthy']:
  if d == "Healthy":
    health_list.append(0)
  else:
    health_list.append(1)

In [208]:
y = np.array(health_list)
y

array([1, 1, 1, ..., 1, 1, 1])

In [209]:
X = dataset.iloc[:,2:]

In [210]:
X.head(5)

,mean_r,mean_g,mean_b,stddev_r,stddev_g,stddev_b,contrast,correlation,inverse_difference_moments,entropy
0,109.290268,120.872437,93.792892,63.903836,51.597123,64.408851,806.727717,0.871684,0.072841,13.924102
1,128.443405,137.412827,114.801193,50.916858,44.833912,58.034531,661.424270,0.850571,0.077148,13.658209
2,105.817230,117.391693,101.533737,63.089894,52.666284,65.218182,225.235099,0.964996,0.155774,12.697456
3,115.003677,126.788773,109.580978,51.001996,41.572264,54.900597,149.451142,0.963721,0.184731,12.172536
4,117.467407,125.006973,106.502304,59.922952,46.281336,60.762472,733.955890,0.861248,0.065945,13.807278


In [211]:
from sklearn.model_selection import train_test_split

In [212]:
train_ratio = 0.70
validation_ratio = 0.10
test_ratio = 0.20

X= shuffle(X)

# test is now 20% of the initial data set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio)

# train is now 70% of the entire data set
# validation is now 10% of the initial data set
x_train,x_val,y_train ,y_val = train_test_split(x_train, y_train, test_size=validation_ratio/(train_ratio + validation_ratio)) 
#print(x_train, x_val, x_test)
print(len(x_train), len(x_val), len(x_test))



1183 170 339


In [213]:
from sklearn.preprocessing import StandardScaler

In [214]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(x_train)
X_val = sc_X.transform(x_val)
X_test = sc_X.transform(x_test)

In [215]:
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [216]:
# training model with different hyperparameters

svm_model = svm.SVC()
svm_model.fit(X_train, y_train)
predictions = svm_model.predict(X_val)


svm_model2 = svm.SVC(C=10, kernel='linear')
svm_model2.fit(X_train, y_train)
predictions2 = svm_model2.predict(X_val)


svm_model3 = svm.SVC(C=10, gamma=0.5, kernel='rbf')
svm_model3.fit(X_train, y_train)
predictions3 = svm_model3.predict(X_val)

print("Val accuracy, no hyperparameters -->" , metrics.accuracy_score(y_val, predictions))
print("Val accuracy, C=100, kernel=linear -->" , metrics.accuracy_score(y_val, predictions2))
print("Val accuracy C=100, gamma=0.5, kernel=rbf -->" , metrics.accuracy_score(y_val, predictions3))

Val accuracy, no hyperparameters --> 0.8117647058823529
Val accuracy, C=100, kernel=linear --> 0.8117647058823529
Val accuracy C=100, gamma=0.5, kernel=rbf --> 0.6882352941176471


In [217]:
#Tuning hyperparameters using GridSearchCV

param_grid = [{'C': [0.1,1, 10, 100], 'gamma': [1,0.5,0.2,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']},
              {'C': [1, 10, 100, 1000], 'kernel': ['linear']}]

In [ ]:
grid = GridSearchCV(svm.SVC(),param_grid,refit=True,verbose=2)
grid.fit(X_train,y_train)

Fitting 5 folds for each of 76 candidates, totalling 380 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=   0.1s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=   0.1s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=   0.1s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=   0.1s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=   0.1s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ...................... C=0.1, gamma=1, kernel=poly, total=   1.5s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ...................... C=0.1, gamma=1, kernel=poly, total=   1.5s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ...................... C=0.1, gamma=1, kernel=poly, total=   1.4s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ...................... C=0.1, gamma=1, kernel=poly, total=   1.7s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ...................... C=0.1, gamma=1, kernel=poly, total=   1.5s
[CV] C=0.1, gamma=1, kernel=sigmoid ..................................
[CV] .

In [ ]:
print("Best estimators " , grid.best_estimator_)
print("Best parameters ", grid.best_params_)

In [199]:
grid_predictions = grid.predict(X_val)

In [ ]:
print("Val accuracy " , metrics.accuracy_score(y_val, grid_predictions))

In [201]:
grid_predictions = grid.predict(X_test)

In [ ]:
print("Accuracy" , metrics.accuracy_score(y_test, grid_predictions))

In [ ]:
def get_leaf_features(photo):
  fixed_size = tuple((256, 256))
  names = ['mean_r','mean_g','mean_b','stddev_r','stddev_g','stddev_b', \
              'contrast','correlation','inverse_difference_moments','entropy']
  df = pd.DataFrame([], columns=names)

  img = cv2.imread(photo)
  img = cv2.resize(img, fixed_size)

  #Preprocessing
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  gs = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
  blur = cv2.GaussianBlur(gs, (25,25),0)
  ret_otsu,im_bw_otsu = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
  kernel = np.ones((50,50),np.uint8)
  closing = cv2.morphologyEx(im_bw_otsu, cv2.MORPH_CLOSE, kernel)

  final_img = img

  #get features
  #Color features
  red_channel = final_img[:,:,0]
  green_channel = final_img[:,:,1]
  blue_channel = final_img[:,:,2]
  blue_channel[blue_channel == 255] = 0
  green_channel[green_channel == 255] = 0
  red_channel[red_channel == 255] = 0
  
  red_mean = np.mean(red_channel)
  green_mean = np.mean(green_channel)
  blue_mean = np.mean(blue_channel)
  
  red_std = np.std(red_channel)
  green_std = np.std(green_channel)
  blue_std = np.std(blue_channel)
  
  #Texture features
  textures = mahotas.features.haralick(gs)
  ht_mean = textures.mean(axis=0)
  contrast = ht_mean[1]
  correlation = ht_mean[2]
  inverse_diff_moments = ht_mean[4]
  entropy = ht_mean[8]
  #write in csv

  vector = [red_mean,green_mean,blue_mean,red_std,green_std,blue_std,\
            contrast,correlation,inverse_diff_moments,entropy ]
  df_temp = pd.DataFrame([vector],columns=names)
  df = df.append(df_temp)
  return df

path = "Test"
files = os.listdir(path)

for file in files:
    photo = os.path.join(path,file)

    leaf_features = get_leaf_features(photo)
   
    if "Healthy" not in photo: 
      y = [1]
    else:
      y = [0];

    y_pred = grid.predict(leaf_features)
    print(photo, y, y_pred)
    #print("Accuracy" , metrics.accuracy_score(y, y_pred ))
